In [ ]:
from abc import ABC, abstractmethod
import random


class Position(ABC):
    
        
    @abstractmethod
    def  __str__(self):
        return f'({self.x}, {self.y})'
    

class Lock:   
class Grid:
    def __init__(self, size):
        self.size = size
        self.grid = [[0 for i in range(size)] for i in range(size)]
        self.blocked_cells = self.generate_blocked_cells()

    def generate_blocked_cells(self):
        num_blocked_cells = int(self.size ** 2 * 0.2)  #bloqueos en el 20% de las celdas
        blocked_cells = set()
        while len(blocked_cells) < num_blocked_cells:
            x, y = random.randint(0, self.size - 1), random.randint(0, self.size - 1)
            blocked_cells.add((x, y))
        return blocked_cells
   




